<a href="https://colab.research.google.com/github/milanschroeder/TRIAS-paper1/blob/main/1x_digitality_zeroshot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
from pprint import pprint

In [ ]:
# make sure to run on GPU!
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# very simple zeroshot classification for benchmarking:
pipeline_zeroshot_classification = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli") # mulitilungual zeroshot classifier, see: https://osf.io/74b8k

In [ ]:
# load classification sample
%pip install pyreadr
import pyreadr
import os
paras_df = pyreadr.read_r('/content/drive/MyDrive/TRIAS/zeroshot_sample.rds')[None]

In [ ]:
import os
output_path='/content/drive/MyDrive/TRIAS/zeroshot_classifications.csv'

In [ ]:
classified_communication = pd.read_csv(output_path) # read already saved classifications (to restart on colab runtime timeot)

In [ ]:
classes = ['digital']
classlabels = ['digital_zeroshot']

In [ ]:
for label in classlabels:
  paras_df[label] = [np.nan] * len(paras_df)

In [ ]:
# define df to classify:
df = paras_df[~paras_df.index.isin(classified_communication.index)].sample(frac=1, random_state=42)

In [ ]:
df # have a look at the data

,id,doc_pos,text_para,doc_key,n_chars_para,doc_type,para_type,digital_zeroshot,DP_zeroshot,lang
2100,1274799.0,10,This is the second Commission antitrust decisi...,detail/en/ip_07_1781,263,ip,paragraph,NaN,NaN,en
2101,1982998.0,9,"What will happen to the cities, which submitte...",detail/en/qanda_22_2592,116,qanda,list,NaN,NaN,en
2102,1460590.0,39,2000/PL/16/P/PT/008 Krakow-Tarnow: road rehabi...,detail/en/ip_00_1357,113,ip,paragraph,NaN,NaN,en
2103,1991750.0,1,Today’s decision sets new rules for how Amazon...,detail/en/ip_22_7777,439,ip,quote,NaN,NaN,en
2104,421352.0,34,The first is social economy and social business.,detail/en/speech_13_61,48,speech,paragraph,NaN,NaN,en
...,...,...,...,...,...,...,...,...,...,...
2179,1884718.0,6,"As regards very high-speed trains, the Commiss...",detail/en/ip_19_881,193,ip,list,NaN,NaN,en
2180,340200.0,22,"Furthermore, we expect to adopt, by the end of...",detail/en/speech_14_2217,357,speech,paragraph,NaN,NaN,en
2181,1370152.0,4,The External Relations Commissioner Chris Patt...,detail/en/ip_03_1582,913,ip,paragraph,NaN,NaN,en
2182,1609495.0,9,"On March 15, the European Parliament adopted t...",detail/en/mex_17_647,1108,mex,paragraph,NaN,NaN,en


Multiple class labels possiblle:

In [ ]:
import time

new_data = df[:0]

# Record the start time
start_time = time.time()
progress = 0

for i, row in df.iterrows():
  output = pipeline_zeroshot_classification(row['text_para'], classes, multi_label = True)
  df.at[i, classlabels[0]], df.at[i, classlabels[1]] = output["scores"]

  new_data = pd.concat([new_data, pd.DataFrame(df.loc[i]).T])

  progress = progress + 1

  if progress % 1000 == 0:
    end_time = time.time()
    mean_time = end_time - start_time
    print(progress, "after", mean_time)

    # save new_data:
    new_data.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False) #append df every 1000 classifications
    new_data = new_data[:0] # truncate again

# save last new_data:
if progress % 1000 != 0:
  new_data.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

# Calculate and print the elapsed time
end_time = time.time()
total_time = end_time - start_time

print("\nTotal time:", total_time, "\nTotal classifications:", progress, "\nmean time per classification:", total_time/progress)


Total time: 73.69004249572754 
Total classifications: 84 
mean time per classification: 0.877262410663423
